In [ ]:
#!pip install keras-unet

In [ ]:
import os
import cv2
import time
import shutil
import random
import inspect
#import imageio as im
import numpy as np
import mahotas as mh
from PIL import Image
from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#from skimage import measure, filters

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
######################################################################################################
experiment = "test2" 
######################################################################################################
real = f"{working_dir}/{experiment}/real_images"
#########################################################################################################

directories = {
    "normalized": f"{working_dir}/{experiment}/normalized_images",
    "cropped": f"{working_dir}/{experiment}/cropped_images",
    "npy": f"{working_dir}/{experiment}/pre_processing/npy",
    "originals": f"{working_dir}/{experiment}/pre_processing/originals",
    "masks": f"{working_dir}/{experiment}/pre_processing/masks",
    "test": f"{working_dir}/{experiment}/pre_processing/test",
    "s": f"{working_dir}/{experiment}/S",
    "crop_original": f"{working_dir}/{experiment}/pre_processing/crop_originals",
    "crop_masks": f"{working_dir}/{experiment}/pre_processing/crop_masks"
}

In [ ]:
import keras
from keras_unet.models import custom_unet

In [ ]:
npy = f"{working_dir}/{experiment}/pre_processing/npy"

In [ ]:
machine = f"{working_dir}/{experiment}/pre_processing/machine"

In [ ]:
images_file = os.path.join(npy, 'original.npy')
labels_file = os.path.join(npy, 'mask.npy')
#test_files = os.path.join(npy, 'test.npy')

In [ ]:
images = np.load(images_file)
labels = np.load(labels_file)
#test = np.load(test_file)

In [ ]:
images.shape

In [ ]:
 # full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)
#test = test.reshape(test.shape[0],test.shape[1],test.shape[2],1)

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
p = np.random.permutation(len(images))

In [ ]:
images = images[p]
labels = labels[p]
#test = test[p]

In [ ]:
labels = labels.astype(np.float64)

In [ ]:
images = images.astype(np.float64)
for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


In [ ]:
images.shape

In [ ]:
X_train = images[0:1200]
y_train = labels[0:1200]
X_val = images[1201:1700]
y_val = labels[1201:1700]
X_test = images[1701:1812]
y_test = labels[1701:1812]

#X_test = test[80:]
#y_test = test[80:]

In [ ]:
y_train.shape

In [ ]:
##########################  MODEL -1######################

In [ ]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=32,
    dropout=0.5,
    output_activation='sigmoid')

In [ ]:
######################################     MODEL - 2#####################

In [ ]:
# from keras import Model
# from keras import regularizers
# from keras.layers import Conv2D

# # Create the custom_unet model
# unet_model = custom_unet(
#     input_shape=(512, 512, 1),
#     use_batch_norm=False,
#     num_classes=1,
#     filters=32,
#     dropout=0.5,
#     output_activation='sigmoid')

# # Add a Conv2D layer with L1 and L2 regularization on top of the custom_unet model
# x = Conv2D(filters=64,
#            kernel_size=(3, 3),
#            kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01),
#            bias_regularizer=regularizers.l2(0.01))(unet_model.output)

# # Create a new model that includes the custom_unet model and the additional layer
# model = Model(inputs=unet_model.input, outputs=x)


In [ ]:
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [ ]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [ ]:
##### batch size 50 , epochs =1000

In [ ]:
# def training(batch,epochs):
#     print ("training and prediction for","batch size:  ",batch, "epochs:  ", epochs)
#     history = model.fit(X_train, 
#                     y_train, 
#                     batch, 
#                     epochs=epochs,
#                     validation_data=(X_val, y_val), 
#                     verbose=1)
    
#     from keras_unet.utils import plot_segm_history

#     plot_segm_history(history)
    
#     y_pred = model.predict(X_test)
    
#     from keras_unet.utils import plot_imgs

#     plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=100)# number to plot also need to e defined in function
    

In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs

def training(batch, epochs):
    print("training and prediction for", "batch size: ", batch, "epochs: ", epochs)

#     # Regularization
#     layer = Dense(units=64,
#                   kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01),
#                   bias_regularizer=regularizers.l2(0.01))

    # Early stopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10)

#     # Data augmentation
#     datagen = ImageDataGenerator(rotation_range=20,
#                                  width_shift_range=0.2,
#                                  height_shift_range=0.2,
#                                  horizontal_flip=True)
#     datagen.fit(X_train)

#     # Training with data augmentation
#     history = model.fit(datagen.flow(X_train, y_train, batch_size=batch),
#                         steps_per_epoch=len(X_train) / batch,
#                         epochs=epochs,
#                         validation_data=(X_val, y_val),
#                         callbacks=[early_stopping])


########################################ONLY EARLY STOPPING###################################
    # Early stopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    # Training with early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val))
                        

    # Plotting results
    plot_segm_history(history)
    y_pred = model.predict(X_test)
    # Apply thresholding
    threshold = 5.0
    y_pred_binary = (y_pred > threshold).astype(np.uint8)
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=100)


In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs

In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs

def trainingearlystopping(batch, epochs):
    print("training and prediction for", "batch size: ", batch, "epochs: ", epochs)
########################################ONLY EARLY STOPPING###################################
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    # Training with early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Plotting results
    plot_segm_history(history)
    y_pred = model.predict(X_test)
    # Apply thresholding
    threshold = 0.2
    y_pred_binary = (y_pred > threshold).astype(np.uint8)
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=100)


In [ ]:
training(60,10)

In [ ]:
training(60,800)

In [ ]:
trainingearlystopping(60,100)

In [ ]:
trainingearlystopping(60,100)

In [ ]:
############ TRYING THRESHOLD VALUES
import matplotlib.pyplot as plt

def trainingwiththres(batch, epochs, thresholds, n_images):
    print("training and prediction for", "batch size: ", batch, "epochs: ", epochs)

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    # Training with early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Plotting results
    plot_segm_history(history)
    y_pred = model.predict(X_test)

    # Apply multiple thresholds
    y_pred_binaries = []
    for threshold in thresholds:
        y_pred_binary = (y_pred > threshold).astype(np.uint8)
        y_pred_binaries.append(y_pred_binary)

    # Plot binary masks for multiple thresholds
    fig, axes = plt.subplots(n_images, len(thresholds), figsize=(15, 15))
    for i in range(n_images):
        for j in range(len(thresholds)):
            axes[i][j].imshow(y_pred_binaries[j][i])
            axes[i][j].set_title(f'Threshold {thresholds[j]}')
            axes[i][j].axis('off')
    plt.show()

In [ ]:
thresholds = [0.2, 0.4, 0.6, 0.8]
n_images = 10
history = trainingwiththres(60, 100, thresholds, n_images)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
from keras.metrics import MeanIoU

n_classes = 2 # number of classes in your segmentation masks
IoU = MeanIoU(num_classes=n_classes)

model = Sequential()
# Encoder
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(512, 512, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

# Decoder
model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
model.add(Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU])

In [ ]:
# Fit the model to the data
history = model.fit(X_train,
                    y_train,
                    batch_size=60,
                    epochs=100,
                    validation_data=(X_val, y_val))

In [ ]:
 # Plotting results
from keras_unet.utils import plot_segm_history

metrics = ['loss', 'val_loss', 'mean_io_u_1', 'val_mean_io_u_1']
plot_segm_history(history, metrics=metrics)
y_pred = model.predict(X_test)
# Apply thresholding

#y_pred_binary = (y_pred > threshold).astype(np.uint8)
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=90)


In [ ]:
def modelFCN(): #Fully Convolutional Network (FCN
  from keras import regularizers
  from keras.models import Sequential
  from keras.layers import Dense
  from keras.callbacks import EarlyStopping
  from keras.preprocessing.image import ImageDataGenerator
  from keras_unet.utils import plot_segm_history, plot_imgs
  from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
  from keras.metrics import MeanIoU

  n_classes = 2 # number of classes in your segmentation masks
  IoU = MeanIoU(num_classes=n_classes)

  model = Sequential()
  # Encoder
  model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(512, 512, 1)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))

  # Decoder
  model.add(Conv2DTranspose(128, (3, 3), strides=(2, 2), activation='relu', padding='same'))
  model.add(Conv2DTranspose(64, (3, 3), strides=(2, 2), activation='relu', padding='same'))
  model.add(Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'))

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[IoU])

In [ ]:
def newtrain(batch,epochs,plot):
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val))
         # Plotting results
    from keras_unet.utils import plot_segm_history

    metrics = ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u']
    plot_segm_history(history, metrics=metrics)
    y_pred = model.predict(X_test)   
    # Apply thresholding
    #y_pred_binary = (y_pred > threshold).astype(np.uint8)
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=plot)

In [ ]:
def nt_early(batch,epochs,plot):
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=20)     
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])
         # Plotting results
    from keras_unet.utils import plot_segm_history

    metrics = ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u']
    plot_segm_history(history, metrics=metrics)
    y_pred = model.predict(X_test)   
    # Apply thresholding
    #threshold = 0.7
    #y_pred = (y_pred > threshold).astype(np.uint8)
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=plot)

In [ ]:
#newtrain(60,4000,150)

In [ ]:
#newtrain(60,400,150)

In [ ]:
def nt_early_pp(batch,epochs,plot,thres):
    import cv2
    from PIL import Image
    # Early stopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=20)     
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val))#,
                        #callbacks=[early_stopping])
    y_pred = model.predict(X_test)
    # Apply thresholding to the predicted images
    threshold = thres # set threshold value
    y_pred = (y_pred > threshold).astype(np.uint8)
    # Apply contrast stretching to the predicted images
    new_min_value = 0
    new_max_value = 255

    for i in range(y_pred.shape[0]):
        #print(y_pred[i].shape)
        #print(y_pred[i].dtype)
        image = Image.fromarray((y_pred[i].squeeze() * 255).astype(np.uint8))
        min_value, max_value = image.getextrema()
        scale = (new_max_value - new_min_value) / (max_value - min_value)
        image = image.point(lambda i: (i - min_value) * scale + new_min_value)
        y_pred[i] = np.expand_dims(np.array(image), axis=-1) / 255
        
    # Plotting results
    from keras_unet.utils import plot_segm_history

    metrics = ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u']
    plot_segm_history(history, metrics=metrics)   
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=plot)
    # Threshold the predicted segmentation mask
    #threshold = 0.5
    #y_pred_binary = (y_pred > threshold).astype(np.uint8)
    
    # Check if the shapes of y_pred_binary and labels are the same
#     print(y_pred_binary.shape)
#     print(X_test.shape)
#     print(y_train.shape)
#     print(X_train.shape)
#     if y_pred_binary.shape != X_test.shape:
#         # Resize labels to match the shape of y_pred_binary
#         labels = cv2.resize(labels, (y_pred_binary.shape[1], y_pred_binary.shape[0]), interpolation=cv2.INTER_NEAREST)
    
#     # Apply connected component analysis to remove small connected components
#     num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(y_pred_binary)

#     # Find the largest connected component (excluding the background label)
#     largest_label = np.argmax(stats[1:, cv2.CC_STAT_AREA]) + 1

#     # Create a new mask containing only the largest connected component
#     y_pred_filtered = np.zeros_like(y_pred_binary)
#     y_pred_filtered[labels == largest_label] = 1

#     # Plot the filtered mask
#     plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred_filtered, nm_img_to_plot=plot)

In [ ]:
nt_early_pp(60,20,50,0.2)

In [ ]:
nt_early_pp(60,20,50,0.4)

In [ ]:
nt_early_pp(60,20,50,0.6)

In [ ]:
nt_early_pp(60,20,50,0.8)

In [ ]:
nt_early_pp(60,20,50,0.7)

In [ ]:
from sklearn.metrics import classification_report
import os
save_dir = machine
# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

def nt_early_pp(batch,epochs,plot,thres):
    import cv2
    from PIL import Image
    
    # Early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val))
    
    # Make predictions and apply thresholding
    y_pred = model.predict(X_test)
    y_pred = (y_pred > thres).astype(np.uint8)
    
    # Calculate precision, recall, and F1 score
    report = classification_report(y_test.flatten(), y_pred.flatten(), target_names=['background', 'circle'], output_dict=True)
    print('Classification report:\n', report)
    
    # Apply contrast stretching to the predicted images
    new_min_value = 0
    new_max_value = 255

    for i in range(y_pred.shape[0]):
        image = Image.fromarray((y_pred[i].squeeze() * 255).astype(np.uint8))
        min_value, max_value = image.getextrema()
        scale = (new_max_value - new_min_value) / (max_value - min_value)
        image = image.point(lambda i: (i - min_value) * scale + new_min_value)
        y_pred[i] = np.expand_dims(np.array(image), axis=-1) / 255
        
    # Plotting results
    from keras_unet.utils import plot_segm_history
    save_path = os.path.join(save_dir, f'image_{i}.png')
    cv2.imwrite(save_path, y_pred)

    metrics = ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u']
    plot_segm_history(history, metrics=metrics)   
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=plot)


In [ ]:
from sklearn.metrics import classification_report

def nt_early_pp(batch,epochs,plot,thres):
    import cv2
    from PIL import Image
    
    # Early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val))
    
    # Make predictions and apply thresholding
    y_pred = model.predict(X_test)
    y_pred = (y_pred > thres).astype(np.uint8)
    
    # Calculate precision, recall, and F1 score
    report = classification_report(y_test.flatten(), y_pred.flatten(), target_names=['background', 'circle'], output_dict=True)
    print('Classification report:\n', report)
    
    # Apply contrast stretching to the predicted images
    new_min_value = 0
    new_max_value = 255

    for i in range(y_pred.shape[0]):
        image = Image.fromarray((y_pred[i].squeeze() * 255).astype(np.uint8))
        min_value, max_value = image.getextrema()
        scale = (new_max_value - new_min_value) / (max_value - min_value)
        image = image.point(lambda i: (i - min_value) * scale + new_min_value)
        y_pred[i] = np.expand_dims(np.array(image), axis=-1) / 255
        
    # Plotting results
    from keras_unet.utils import plot_segm_history

    metrics = ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u']
    plot_segm_history(history, metrics=metrics)   
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=plot)


In [ ]:
nt_early_pp(60,200,50,0.2)

In [ ]:
nt_early_pp(60,200,50,0.3)

In [ ]:
nt_early_pp(60,200,50,0.4)

In [ ]:
nt_early_pp(60,200,50,0.8)

In [ ]:
nt_early_pp(60,200,50,0.6)

In [ ]:
nt_early_pp(60,200,50,0.9)

In [ ]:
nt_early_pp(60,800,100,0.8)

In [ ]:
nt_early_pp(60,2,5)

In [ ]:
nt_early(60,400,100)

In [ ]:
history = model.fit(X_train,
                        y_train,
                        batch_size=60,
                        epochs=4,
                        validation_data=(X_val, y_val))
         # Plotting results
from keras_unet.utils import plot_segm_history

metrics = ['loss', 'val_loss', 'mean_io_u', 'val_mean_io_u']
plot_segm_history(history, metrics=metrics)
y_pred = model.predict(X_test)   
# Apply thresholding
threshold = 0.9
y_pred_binary = (y_pred > threshold).astype(np.uint8)
plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=5)

In [ ]:
history.history.keys()

In [ ]:
import matplotlib.pyplot as plt

# Plot training history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

# Make predictions
y_pred = model.predict(X_test)

# Apply thresholding
threshold = 5.0
y_pred_binary = (y_pred > threshold).astype(np.uint8)

# Plot segmentation results
n = 3 # number of images to plot
for i in range(n):
    plt.subplot(1, n, i+1)
    plt.imshow(y_pred_binary[i,:,:,0], cmap='gray')
    plt.axis('off')
plt.show()

In [ ]:
#training(60,100)

In [ ]:

# a little overlifting has been noticed in the above graph so we are using new techiniques to decrease that

In [ ]:
training(60,100)

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    50, 
                    epochs=1000,
                    validation_data=(X_val, y_val), 
                    verbose=1)

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

In [ ]:
training(60,40)

In [ ]:
#help()

In [ ]:
training(65,65)

In [ ]:
overlay(directories['originals'],directories['masks'], 5)

In [ ]:
training(60,4000)

In [ ]:
training(64,20)

In [ ]:
import os
import random
from PIL import Image

def overlay(original_dir, mask_dir, n):
    original_images = os.listdir(original_dir)
    mask_images = os.listdir(mask_dir)

    for i in range(n):
        original_image = random.choice(original_images)
        mask_image = original_image.replace('.png', '_mask.png')
        if mask_image in mask_images:
            img = Image.open(os.path.join(original_dir, original_image))
            mask = Image.open(os.path.join(mask_dir, mask_image))
            img.paste(mask, (0, 0), mask)
            img.show()
            #hint: # overlay_masks('original', 'masks', 3) - To plot overlay wiht masks



In [ ]:
# !pip install keras-unet

# import keras
# from keras_unet.models import custom_unet

# images_file = os.path.join(npy_files, 'original.npy')
# labels_file = os.path.join(npy_files, 'mask.npy')

# images = np.load(images_file)
# labels = np.load(labels_file)

# images.shape

# # full dataset does not have the last channel
# images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
# labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

# images.shape

# p = np.random.permutation(len(images))

# images = images[p]
# labels = labels[p]

# labels = labels.astype(np.float64)

# images = images.astype(np.float64)

# for i in range(images.shape[0]):
#     images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


# images.shape

# X_train = images[0:50]
# y_train = labels[0:50]
# X_val = images[10:20]
# y_val = labels[10:20]
# X_test = images[0:]
# y_test = labels[0:]

# model = custom_unet(
#     input_shape=(512, 512, 1),
#     use_batch_norm=False,
#     num_classes=1,
#     filters=32,
#     dropout=0.5,
#     output_activation='sigmoid')

# import keras.optimizers
# from keras_unet.metrics import iou, iou_thresholded
# from keras_unet.losses import jaccard_distance



# opt = keras.optimizer_v1.Adam(lr=0.01)

# model.compile(optimizer = 'Adam',    
#               loss='binary_crossentropy', 
#               metrics=[iou, iou_thresholded])

# history = model.fit(X_train, 
#                     y_train, 
#                     50, 
#                     epochs=500,
#                     validation_data=(X_val, y_val), 
#                     verbose=1)

# from keras_unet.utils import plot_segm_history

# plot_segm_history(history)

# y_pred = model.predict(X_test)

# from keras_unet.utils import plot_imgs

# plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

In [ ]:
# My next idea in the pipe line is to compare same images on different iterations and come to conclusion.
# To decide what batch size and epochs gives the best results.